## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [6]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [7]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [19]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 간단하게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

In [20]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 간단하게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x127e18890>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x12760f190>, root_client=<openai.OpenAI object at 0x12760b6d0>, root_async_client=<openai.AsyncOpenAI object at 0x127fd0850>, model_name='gpt-4.1-nano', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [16]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [12]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input).content

"인공지능 모델의 학습 원리는 데이터를 이용해 모델이 문제를 해결하는 방법을 배우는 과정입니다. 일반적으로 다음과 같은 단계로 이루어집니다:\n\n1. **데이터 수집:** 모델이 학습할 수 있도록 많은 예제 데이터를 준비합니다.\n2. **모델 설계:** 문제에 맞는 인공지능 구조(예: 신경망)를 만듭니다.\n3. **학습 과정:** 데이터를 모델에 입력하고, 모델이 내린 예측과 실제 정답을 비교하여 오차(손실)를 계산합니다.\n4. **파라미터 조정:** 오차를 줄이기 위해 모델 내부의 가중치(파라미터)를 조정하는 과정을 반복합니다. 이때 주로 '경사 하강법' 같은 최적화 알고리즘을 사용합니다.\n5. **검증 및 개선:** 학습이 잘 되는지 검증 데이터를 통해 확인하고, 필요하면 모델 구조나 학습 방법을 조정합니다.\n\n이 과정을 통해 모델은 데이터를 이해하고, 새로운 데이터에 대해서도 적절한 예측을 할 수 있게 됩니다."

아래는 스트리밍을 출력하는 예시 입니다.

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 출력파서(Output Parser)


In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [21]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [22]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

"인공지능 모델의 학습 원리는 데이터를 이용해 모델이 문제를 해결하는 방법을 배우는 과정입니다. 일반적으로 다음과 같은 단계로 이루어집니다:\n\n1. **데이터 수집:** 모델이 학습할 수 있도록 많은 예제와 정보를 모읍니다.\n2. **모델 초기화:** 무작위로 또는 기본값으로 시작하는 모델의 가중치(파라미터)를 설정합니다.\n3. **학습 과정:** 모델이 입력 데이터를 받고, 예측을 수행합니다. 그 후, 정답과의 차이(오차)를 계산합니다.\n4. **오차 수정:** 오차를 줄이기 위해 가중치를 조정하는데, 이를 '학습' 또는 '훈련'이라고 합니다. 이 과정은 보통 '경사 하강법' 같은 최적화 알고리즘을 사용합니다.\n5. **반복:** 여러 번 데이터를 반복 학습하면서 모델이 점점 더 정확하게 예측할 수 있도록 개선됩니다.\n\n이 과정을 통해 인공지능 모델은 주어진 문제에 대해 더 잘 대응할 수 있도록 '학습'하게 되는 것입니다."

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [23]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [24]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [25]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
Waiter: Hello! Welcome to our restaurant. Are you ready to order?
Customer: Yes, I’d like to order the grilled chicken salad, please.
Waiter: Certainly. Would you like anything to drink?
Customer: Just water, please. And could I get the dressing on the side?
Waiter: Of course. Anything else?
Customer: No, that’s all. Thank you.
Waiter: You're welcome. Your order will be ready shortly.

- 한글 해석:
웨이터: 안녕하세요! 식당에 오신 것을 환영합니다. 주문하시겠어요?
고객: 네, 그릴드 치킨 샐러드로 주세요.
웨이터: 알겠습니다. 음료는 무엇으로 하시겠어요?
고객: 그냥 물로 주세요. 그리고 드레싱은 따로 주실 수 있나요?
웨이터: 물론입니다. 또 필요하신 게 있으세요?
고객: 아니요, 그게 다예요. 감사합니다.
웨이터: 천만에요. 곧 주문이 준비될 겁니다.


In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [26]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
Customer: Hi, I’d like to order a large pepperoni pizza, please.  
Delivery person: Sure! Would you like anything else?  
Customer: Yes, could you also add a two-liter bottle of cola?  
Delivery person: Absolutely. That’ll be $25.50. Can I get your address?  
Customer: It’s 123 Maple Street, apartment 4B.  
Delivery person: Thanks! Your order will arrive in about 30 minutes.  

- 한글 해석:  
고객: 안녕하세요, 큰 페퍼로니 피자를 주문하고 싶어요.  
배달원: 네! 혹시 다른 것도 주문하시겠어요?  
고객: 네, 콜라 두 리터도 하나 더 주세요.  
배달원: 알겠습니다. 총 금액은 25.50달러입니다. 주소를 알려주시겠어요?  
고객: 메이플 스트리트 123번지, 4B 아파트입니다.  
배달원: 감사합니다! 주문은 약 30분 후에 도착할 예정입니다.